<a href="https://colab.research.google.com/github/Alys9999/AWS-notebook/blob/master/CNN-small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt


In [6]:
# Define paths
dataset_zip = r'/content/drive/MyDrive/NEU/archive.zip'  # Update with your zip file path
dataset_dir = r'/content/drive/MyDrive/NEU/neu-defect-dataset'

# Extract the dataset
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

# Check extracted files
print("Extracted dataset structure:")
!ls -R $dataset_dir


Extracted dataset structure:
/content/drive/MyDrive/NEU/neu-defect-dataset:
NEU-DET

/content/drive/MyDrive/NEU/neu-defect-dataset/NEU-DET:
train  validation

/content/drive/MyDrive/NEU/neu-defect-dataset/NEU-DET/train:
annotations  images

/content/drive/MyDrive/NEU/neu-defect-dataset/NEU-DET/train/annotations:
crazing_100.xml    inclusion_20.xml	   pitted_surface_101.xml   rolled-in_scale_20.xml
crazing_101.xml    inclusion_210.xml	   pitted_surface_102.xml   rolled-in_scale_210.xml
crazing_102.xml    inclusion_211.xml	   pitted_surface_103.xml   rolled-in_scale_211.xml
crazing_103.xml    inclusion_212.xml	   pitted_surface_104.xml   rolled-in_scale_212.xml
crazing_104.xml    inclusion_213.xml	   pitted_surface_105.xml   rolled-in_scale_213.xml
crazing_105.xml    inclusion_214.xml	   pitted_surface_106.xml   rolled-in_scale_214.xml
crazing_106.xml    inclusion_215.xml	   pitted_surface_107.xml   rolled-in_scale_215.xml
crazing_107.xml    inclusion_216.xml	   pitted_surface_108.xml   

In [7]:
train_images_dir = os.path.join(dataset_dir, 'NEU-DET/train/images')
validation_images_dir = os.path.join(dataset_dir, 'NEU-DET/validation/images')

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Generate batches of images
train_generator = train_datagen.flow_from_directory(
    train_images_dir,
    target_size=(128, 128),            # Resize images to 128x128
    batch_size=32,                     # Set batch size
    class_mode='categorical'           # Multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_images_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

Found 1440 images belonging to 6 classes.
Found 360 images belonging to 6 classes.


In [11]:
model = Sequential([
    Input(shape=(128, 128, 3)),             # Use Input layer instead of input_shape
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')          # Update number of classes based on your dataset
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 307ms/step - accuracy: 0.2026 - loss: 2.3093 - val_accuracy: 0.3466 - val_loss: 1.5260
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 280ms/step - accuracy: 0.3895 - loss: 1.4455 - val_accuracy: 0.4602 - val_loss: 1.1993
Epoch 3/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 21s 282ms/step - accuracy: 0.4985 - loss: 1.1811 - val_accuracy: 0.3864 - val_loss: 1.4830
Epoch 4/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 14s 317ms/step - accuracy: 0.6254 - loss: 0.9433 - val_accuracy: 0.3239 - val_loss: 2.2340
Epoch 5/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 19s 283ms/step - accuracy: 0.6490 - loss: 0.8894 - val_accuracy: 0.5369 - val_loss: 1.1754
Epoch 6/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 276ms/step - accuracy: 0.7244 - loss: 0.7440 - val_accuracy: 0.5057 - val_loss: 1.3234
Epoch 7/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 267ms/step - accuracy: 0.8163 - loss: 0.5820 - val_accuracy: 0.6989 - val_loss: 0.8225
Epoch 8/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 12s 259ms/step - accuracy: 0.8444 - loss: 0.4878 - val_accu

In [13]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy:.2f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 166ms/step - accuracy: 0.7084 - loss: 0.8157
Validation Accuracy: 0.73


In [15]:
model.save('/content/drive/MyDrive/models/neu_defect_cnn_model.keras')


In [17]:
# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/models/neu_defect_cnn_model.keras')

# Prepare an image for inference
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to an image for testing
test_image_path = '/content/drive/MyDrive/NEU/neu-defect-dataset/NEU-DET/validation/images/scratches/scratches_300.jpg'

# Load and preprocess the image
img = load_img(test_image_path, target_size=(128, 128))
img_array = img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Perform inference
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
class_labels = list(train_generator.class_indices.keys())

print(f"Predicted class: {class_labels[predicted_class]} with confidence {predictions[0][predicted_class]:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
Predicted class: scratches with confidence 0.37
